# WasteNet

## Setup

In [1]:
# Colab
!rm -r smart-cities-drl
!git clone https://github.com/eescriba/smart-cities-drl
!cd smart-cities-drl/ && pip install -e .

import sys
sys.path.insert(0,'./smart-cities-drl/src/')

# Local
# !pip install -e ..
# import sys
# sys.path.insert(0,'../src/')

In [ ]:
import json
import shutil
import random

import gym
import ray
from ray.tune import run, choice
from core.rl import PPOAgent
from core.pbt import PbtOptimizer
from wastenet.env import WasteNetEnv

## Proximal Policy Optimization (PPO)

### Tune hyperparameters

In [6]:
ppo = PPOAgent("wastenet_ppo_tune", WasteNetEnv, {})

2021-06-24 10:55:53,793	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265
(pid=3548) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/internal/variadic_reduce.py:115: calling function (from tensorflow.python.eager.def_function) with experimental_compile is deprecated and will be removed in a future version.
(pid=3548) Instructions for updating:
(pid=3548) experimental_compile is deprecated, use jit_compile instead
(pid=3547) WARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/tensorflow_probability/python/internal/variadic_reduce.py:115: calling function (from tensorflow.python.eager.def_function) with experimental_compile is deprecated and will be removed in a future version.
(pid=3547) Instructions for updating:
(pid=3547) experimental_compile is deprecated, use jit_compile instead
2021-06-24 10:56:05,651	WARNING util.py:53 -- Install gputil for GPU system monitoring.


In [7]:
tune_config = {
    "env": WasteNetEnv,
    "seed": 123,
    "num_gpus": 1,
    "num_workers": 1,
    "observation_filter": "MeanStdFilter",
    "model": {
        "free_log_std": True
    },
    "lambda": 0.95,
    "clip_param": 0.3,
    "lr": 5e-5,
    "num_sgd_iter": choice([10, 20, 30]),
    "sgd_minibatch_size": choice([128, 512, 2048]),
    "train_batch_size": choice([10000, 20000, 40000])
}
stop_criteria = {
    # "timesteps_total": 5000000
}
hyperparam_mutations={
    "lambda": lambda: random.uniform(0.9, 1.0),
    "clip_param": lambda: random.uniform(0.1, 0.5),
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "num_sgd_iter": lambda: random.randint(1, 30),
    "sgd_minibatch_size": lambda: random.randint(128, 16384),
    "train_batch_size": lambda: random.randint(2000, 160000),
}
pbt = PbtOptimizer(hyperparam_mutations)

In [ ]:
ppo.restart()
analysis = ppo.tune(tune_config, stop_criteria, scheduler=pbt.scheduler, num_samples=4)
best_config =  analysis.get_best_config(metric="episode_reward_mean", mode="max")
print("Best hyperparameters found: ", best_config)

In [ ]:
best_config = {
    "observation_filter": "MeanStdFilter",
    "model": {"free_log_std": True},
    "num_sgd_iter": 10,
    "sgd_minibatch_size": 128,
    "lambda": 0.731396,
    "clip_param": 0.317651,
    "lr": 5e-05,
    "train_batch_size": 18812,
}

### Training

In [ ]:
ppo = PPOAgent("wastenet_ppo_train", best_config, WasteNetEnv, {})
ppo.train(num_iter=200)

In [ ]:
policy = ppo.agent.get_policy()
model = policy.model
print(model.base_model.summary())

### Testing

In [ ]:
# ppo = PPOAgent("wastenet_ppo_test", best_config, WasteNetEnv, {})
# ppo.load("checkpoints/checkpoint-best")
ppo.test(num_episodes=1000)

### Visualization

In [ ]:
!zip -r /root/ray_results.zip /root/ray_results

In [ ]:
%load_ext tensorboard 
%tensorboard --logdir="/root/ray_results/"